In [ ]:
pip install keras==2.4.3

In [ ]:
pip install tensorflow==2.4.1

In [ ]:
pip install keras==2.1.0

In [ ]:
pip install tensorflow==1.13.1

In [ ]:
pip install tensorflow-gpu==1.13.1

In [ ]:
pip install scikit-image==0.16.2

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
path='/content/gdrive/"My Drive"/"Colab Notebooks"/"Label"'
%ls{path}

In [ ]:
!cp /content/gdrive/MyDrive/Colab\ Notebooks/mrcnn/model.py /content
!cp /content/gdrive/MyDrive/Colab\ Notebooks/mrcnn/utils.py /content
!cp /content/gdrive/MyDrive/Colab\ Notebooks/mrcnn/visualize.py /content
!cp /content/gdrive/MyDrive/Colab\ Notebooks/mrcnn/config.py /content
!cp /content/gdrive/MyDrive/Colab\ Notebooks/mrcnn/__init__.py /content

In [ ]:
import keras
import tensorflow
print(keras.__version__)
print(tensorflow.__version__)


In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils

WEIGHTS_PATH ='/content/gdrive/MyDrive/Colab Notebooks/mask_rcnn_coco.h5'

In [ ]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"

    IMAGES_PER_GPU = 2

    NUM_CLASSES = 1 + 1  # Background + phone,laptop and mobile

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 100

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

In [ ]:
config = CustomConfig()
CUSTOM_DIR = path
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.9

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [ ]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "gesture")
        assert subset in ["train", "test"]
        dataset_dir = os.path.join(dataset_dir, subset)
        print(dataset_dir)
        if(subset=="train"):
          annotations1 = json.load(open(dataset_dir+'/train.json'))
        else:
          annotations1 = json.load(open(dataset_dir+'/test.json'))
        annotations = list(annotations1.values())  # don't need the dict keys
        annotation = [a for a in annotations if a['regions']]
        
        # Add images
        for a in annotations:
           
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"gesture": 1}
            num_ids = [name_dict[a] for a in objects]

            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object", 
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
       
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):

        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_custom('/content/gdrive/MyDrive/Colab Notebooks/Label', "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_custom('/content/gdrive/MyDrive/Colab Notebooks/Label', "test")
    dataset_val.prepare()
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=55,
                layers='heads')


In [ ]:
DEFAULT_LOGS_DIR = os.path.join('/content/gdrive/MyDrive/Colab Notebooks', "logs")
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,model_dir=DEFAULT_LOGS_DIR)


if not os.path.exists(WEIGHTS_PATH):
  utils.download_trained_weights(WEIGHTS_PATH)

model.load_weights(WEIGHTS_PATH, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)			

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
/content/gdrive/MyDrive/Colab Notebooks/Label/train
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numids [1]
objects: ['gesture']
numid

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2039: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/55
 99/100 [============================>.] - ETA: 2s - loss: 1.7255 - rpn_class_loss: 0.0175 - rpn_bbox_loss: 0.4613 - mrcnn_class_loss: 0.0709 - mrcnn_bbox_loss: 0.6181 - mrcnn_mask_loss: 0.5577

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2197: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


100/100 [==============================] - 403s 4s/step - loss: 1.7188 - rpn_class_loss: 0.0174 - rpn_bbox_loss: 0.4583 - mrcnn_class_loss: 0.0711 - mrcnn_bbox_loss: 0.6161 - mrcnn_mask_loss: 0.5558 - val_loss: 1.1609 - val_rpn_class_loss: 0.0121 - val_rpn_bbox_loss: 0.3514 - val_mrcnn_class_loss: 0.0530 - val_mrcnn_bbox_loss: 0.4316 - val_mrcnn_mask_loss: 0.3128
Epoch 2/55
100/100 [==============================] - 304s 3s/step - loss: 1.0126 - rpn_class_loss: 0.0104 - rpn_bbox_loss: 0.3035 - mrcnn_class_loss: 0.0424 - mrcnn_bbox_loss: 0.3786 - mrcnn_mask_loss: 0.2777 - val_loss: 0.9463 - val_rpn_class_loss: 0.0080 - val_rpn_bbox_loss: 0.2955 - val_mrcnn_class_loss: 0.0346 - val_mrcnn_bbox_loss: 0.3400 - val_mrcnn_mask_loss: 0.2683
Epoch 3/55
100/100 [==============================] - 301s 3s/step - loss: 0.9879 - rpn_class_loss: 0.0093 - rpn_bbox_loss: 0.3525 - mrcnn_class_loss: 0.0336 - mrcnn_bbox_loss: 0.3384 - mrcnn_mask_loss: 0.2540 - val_loss: 0.8105 - val_rpn_class_loss: 0.0065